# Databricks Notebook: Smart Home Energy ETL
# Week 4 - Databricks ETL for Smart Energy Monitoring

# -------------------------------------------------------
# Step 1: Setup widgets (so you can change file paths easily)
# -------------------------------------------------------

In [ ]:

dbutils.widgets.text("input_path", "/FileStore/tables/energyusage.csv")
dbutils.widgets.text("delta_db", "smart_energy_db")
dbutils.widgets.text("out_dir", "/FileStore/tables/smart_energy_outputs")
dbutils.widgets.text("alert_threshold_kwh", "10")

INPUT_PATH = dbutils.widgets.get("input_path")
DELTA_DB   = dbutils.widgets.get("delta_db")
OUT_DIR    = dbutils.widgets.get("out_dir")
ALERT_KWH  = float(dbutils.widgets.get("alert_threshold_kwh"))

In [ ]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {DELTA_DB}")
dbutils.fs.mkdirs(OUT_DIR)

In [ ]:

# Step 3: Read cleaned logs from CSV

from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, TimestampType
from pyspark.sql.functions import col

schema = StructType([
    StructField("device_id", IntegerType(), True),
    StructField("room_id", IntegerType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("energy_kwh", DoubleType(), True)
])

# Load CSV
df = (spark.read
      .option("header", True)
      .schema(schema)
      .csv(INPUT_PATH))

# Clean: drop nulls and invalid readings
df_clean = (df
    .dropna(subset=["device_id","room_id","timestamp","energy_kwh"])
    .filter(col("energy_kwh") > 0)
)


In [ ]:

# Step 4: Add day/week columns for aggregation

from pyspark.sql.functions import to_date, date_trunc

df_enriched = (df_clean
    .withColumn("day", to_date(col("timestamp")))
    .withColumn("week", date_trunc("week", col("timestamp")))
)


In [ ]:
# Step 5: Daily summary (sum, avg, count)

from pyspark.sql.functions import sum as s_sum, avg as s_avg, count as s_count

daily = (df_enriched
    .groupBy("day","device_id","room_id")
    .agg(
        s_sum("energy_kwh").alias("daily_kwh"),
        s_avg("energy_kwh").alias("daily_avg_kwh"),
        s_count("*").alias("readings")
    )
)

# Save results in Delta + CSV
daily.write.mode("overwrite").format("delta").saveAsTable(f"{DELTA_DB}.daily_usage")
daily.coalesce(1).write.mode("overwrite").option("header", True).csv(f"{OUT_DIR}/daily_usage_csv")

In [ ]:

# Step 6: Weekly summary (sum, avg, count)

weekly = (df_enriched
    .groupBy("week","device_id","room_id")
    .agg(
        s_sum("energy_kwh").alias("weekly_kwh"),
        s_avg("energy_kwh").alias("weekly_avg_kwh"),
        s_count("*").alias("readings")
    )
)

# Save results in Delta + CSV
weekly.write.mode("overwrite").format("delta").saveAsTable(f"{DELTA_DB}.weekly_usage")
weekly.coalesce(1).write.mode("overwrite").option("header", True).csv(f"{OUT_DIR}/weekly_usage_csv")

In [ ]:

# Step 7: Peak vs Off-Peak usage classification

from pyspark.sql.functions import hour, when

classified = (df_enriched
    .withColumn("hour", hour(col("timestamp")))
    .withColumn("usage_type", when((col("hour") >= 18) & (col("hour") <= 23), "peak").otherwise("off_peak"))
)

# Pivot to show peak/off-peak usage side by side
pv = (classified
    .groupBy("device_id","usage_type")
    .agg(s_sum("energy_kwh").alias("total_kwh"))
    .groupBy("device_id")
    .pivot("usage_type", ["peak","off_peak"]).sum("total_kwh")
    .na.fill(0.0)
    .withColumn("total_usage", col("peak")+col("off_peak"))
)

# Save results
pv.write.mode("overwrite").format("delta").saveAsTable(f"{DELTA_DB}.peak_offpeak_usage")
pv.coalesce(1).write.mode("overwrite").option("header", True).csv(f"{OUT_DIR}/peak_offpeak_usage_csv")

In [ ]:

# Step 8: Alerts for devices exceeding daily threshold

alerts = spark.sql(f'''
  SELECT day, device_id, room_id, daily_kwh
  FROM {DELTA_DB}.daily_usage
  WHERE daily_kwh > {ALERT_KWH}
  ORDER BY daily_kwh DESC
''')

# Save alerts to CSV for dashboard/notifications
alerts.coalesce(1).write.mode("overwrite").option("header", True).csv(f"{OUT_DIR}/alerts_csv")
